
<img src="img/logo_wiwi.png" width="25%" align="left">

<img src="img/decision_analytics_logo.png" width="17%" align="right">



<br><br><br><br><br><br><br><br>



# Algorithmen und Datenstrukturen(A+D)-Projekt 

**Sommersemester 2022**


# 4. Rollout-Varianten

<br>

<br>
<br>

**J-Prof. Dr. Michael Römer, Till Porrmann, Jakob Schulte, Henning Witteborg**

Juniorprofessur für Decision Analytics  | Universität Bielefeld

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numba import njit

## Was machen wir heute?

**Organisatorisch:**
- Erste Eindrücke vom Projekt
- Zuteilung der Projektbetreuer

**Inhaltlich:**
- Rollout, etwas anders implementiert und interpretiert
- Multi-Start Rollout
- Rollout mit Multistep Lookahead





# Organisatorisches

## Erste Eindrücke aus der Projektarbeit

- Konnten Sie sich mit dem Thema vertraut machen?
- Haben Sie bereits ein Greedy-Verfahren gefunden?
  - ggf. schon erste Implementierungsversuche?
- Haben Sie Instanzen gefunden?
  - wenn ja, können Sie diese bereits einlesen?
- Haben Sie Fragen / Probleme, die wir hier diskutieren können?
  

## Betreuer der Projektgruppen:

|Gruppe | Thema | Gruppenmitglieder | Betreuer
|-|:-|:-|:-
|1| Set Covering | SA, EB, HC, KB, HH | Henning Witteborg
|2| Set Covering | GK, FD, GF, FE, AA | Henning Witteborg
|3| Maximum Independent Set | RF, MF, SH, SR | Michael Römer
|4| Maximum Independent Set | AE, KS, DS | Michael Römer
|5| Vertex Coloring | JF, LM, IT, JS | Till Porrmann
|6| Vertex Coloring | AH, NK, NS, DP | Till Porrmann
|7| Job Shop Scheduling | JB, JR, CG, PH | Jakob Schulte
|8| Job Shop Scheduling| LF, LH, FS, SK | Jakob Schulte

- Vereinbaren Sie gern per E-Mail / nach der Veranstaltung einen ersten "Kennenlerntermin"!

# Inhaltlicher Teil


## Das Python-Paket `python-tsp`

siehe: https://github.com/fillipe-gsm/python-tsp

### bietet:
- Funktionen zum Einlesen von TSP-Instanzen im tsplib-format
  
  
  

In [12]:
from python_tsp.distances import tsplib_distance_matrix

#tsplib_file = "./../problems/tsp/instances/a280.tsp" # optimale Lösung 2579 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
#tsplib_file = "./../problems/tsp/instances/brazil58.tsp" # optimale Lösung 25395 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)
tsplib_file = "./../problems/tsp/instances/berlin52.tsp" # optimale Lösung 7542 (lt. http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/STSP.html)

distance_matrix = tsplib_distance_matrix(tsplib_file)

## Der Nearest-Neighbor-Algortihmus als Beispiel für ein Greedy-Verfahren

<img src="./img/32.png" width="20%" align="right">


Eine einfache und sehr bekannte Heuristik für das SHPP (und das TSP) nennt sich **nearest neighbor**:
- man startet bei einem Knoten und
- wählt in jedem Schritt immer den Knoten aus, der dem aktuellen am nächsten ist:

Beispiel rechts: wir starten in Marin und suchen immer den nächsten Nachbarn

## Nearest Neighor in Python: Überblick

**Wie funktioniert der Algorithmus?**

**Eingabe:** Distanzmatrix, Startknoten

- aktueller Knoten `node` := Startknoten
- Solange noch nicht alle Knoten besucht wurden:
  - wähle den **noch nicht besuchten** Knoten **mit kürzester Distanz** von `node`
  - füge den Knoten zur Tour (Liste der besuchten Knoten) hinzu

**Die Länge der Tour** berechnet sich dann als:
- Distanz zwischen jedem Knoten und Nachfolger in der Tour
    - wobei der Nachfolger des letzen Knotens der Startknoten ist
- diese Berechnung kann natürlich "on-the-fly" im Nearest-Neighbor-Algorithmus berechnet werden

## Hilfsfunktion: `select_nearest_neighbor`

- wir ändern / vereinfachen unsere bisherige Implementierung der Hilfsfunktion
- wir brauchen eigentlich neben der Distanzmatrix nur die Permutationsliste
  - das letzte Element ist der Knoten, von dem aus gesucht wird
- wir nutzen wieder `njit` von numba (könnte man auch weglassen, wäre dann aber langsamer)

In [356]:
@njit
def select_nearest_neighbor(permutation, distance_matrix ):
    
    # node ist der letzte Knoten der Permutation
    # wir könnten auch permutation[-1] schreiben, aber das funktioniert mit Numba nicht
    node = permutation[len(permutation)-1]
    
    smallest_distance = 9999999999 ## grosser Wert
    nearest_neighbor = 0
    
    #Anzahl an Knoten = Dimension der Distanzmatrix
    for neighbor in range(len(distance_matrix)):
        
        if neighbor in permutation:
            continue
            
        if distance_matrix[node][neighbor] < smallest_distance:
            nearest_neighbor = neighbor
            smallest_distance = distance_matrix[node][neighbor]            
       
    return nearest_neighbor, smallest_distance 

## Implementierung des Nearest-Neighbor-Algorithmus
- wir nutzen nun die neue Funktion in der Implementierung des Nearest Neighbor-Algorithmus
- auch hier haben wir die Implementierung etwas vereinfacht:
  - wir übergeben nur eine Permutation und die Distanzmatrix

- beim alleinigen Aufruf des Algorithmus übergeben wir eine Liste mit einem Startknoten
- bei Verwendung im Rollout-Algorithmus übergeben wir die bisherige Permutatoin


In [357]:
@njit
def tsp_nearest_neighbor(permutation, distance_matrix ):
    
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(permutation) < len(distance_matrix):
        
        node, distance = select_nearest_neighbor(permutation, distance_matrix )
        
        permutation.append(node)
        total_distance += distance
        
    total_distance += distance_matrix[permutation[len(permutation)-1], permutation[0]]
    return permutation, total_distance


..probieren wir es aus:

In [360]:
permutation, distance = tsp_nearest_neighbor([0], distance_matrix)
print(distance)

8980


## Eine Routine zum Evaluieren einer Lösung

- immer wenn man nicht-triviale Algorithmen für Optimierungsprobleme entwickelt, sollte man einen "Solution-Checker" schreiben / nutzen

**Im Fall des TSP gilt es:**
- zu prüfen, ob
  - die Lösung die Richtige Anzahl an Knoten enthält
  - dass es sich bei der Lösung tatsächlich um eine Permutation der Indizes handelt (kein Index kommt zweimal vor)
- die Distanz der Tour zu berechnen

In [27]:
def evaluate_tsp_solution(distance_matrix, permutation):
    n = len(distance_matrix)
    if len(permutation) != n:
        print ("Wrong number of nodes")
        return -1
     # Menge der Lösungsindizes muss = der Menge der Indizes von 0 bis n-1 sein
    if set(permutation) != set(range(n)):
        print ("Not a proper permutation!")
        return  -1
    
    total_distance = 0
    for i in range(n):
        if i < n-1:
            total_distance += distance_matrix[permutation[i],permutation[i+1]]
        else:
            total_distance += distance_matrix[permutation[i],permutation[0]]   
            
    return total_distance    

## Erweiterung: Evaluation zum Überprüfen einer gegebenen Distanz

- oftmals ist es nützlich, in einer Funktion direkt die vom Algorithmus berechnete Distanz zu prüfen
- folgende Funktion macht eine entsprechende Ergebnisausgabe:


In [28]:
def print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance):
    
    eval_distance = evaluate_tsp_solution(distance_matrix, permutation)
    
    if distance == eval_distance:
        print ("Solution feasible, distance is: ", distance)
    elif eval_distance < 0:
        print("Solution infeasible")
    else: 
        print("Solution feasible, wrong distance: ", distance, " evaluation gave ", eval_distance)

In [360]:
permutation, distance = tsp_nearest_neighbor([0], distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance)

8980


## Multistart-Nearest Neighbor

- wir hatten gesehen, dass es sich lohnen kann, an allen möglichen Knoten zu starten:

In [361]:
def tsp_multistart_nearest_neighbor(distance_matrix):
    best_permutation = []
    best_distance = 999999
    
    for i in range(len(distance_matrix)):
        permutation, distance = tsp_nearest_neighbor([i], distance_matrix)
        
        if distance < best_distance:
            best_permutation = permutation
            best_distance = distance
    
    return best_permutation, best_distance
    

In [362]:
permutation, distance = tsp_multistart_nearest_neighbor( distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance)

Solution feasible, distance is:  8181


## Nearest Neighbor vs Rollout
- beim Nearest Neighbor wählen wir als nächsten Knoten den mit der kürzesten Distanz zum letzten Knoten in der partiellen Tour:

<img src="./img/single_trajectory.png" width="50%">

- beim Rollout wählen wir den mit dem kleinsten Q-Faktor, bestehend aus der Summe von
  - Distanz zum letzen Knoten
  - Länge der Rest-Tour vom letzten Knoten aus

<img src="./img/rollout_general.png" width="50%">

## Rollout, eine neue Implementierung
- beim Nearest Neighbor wählen wir als nächsten Knoten den mit der kürzesten Distanz zum letzten Knoten in der partiellen Tour
- beim Rollout wählen wir den mit dem kleinsten Q-Faktor, bestehend aus der Summe von
  - Distanz zum letzen Knoten
  - Länge der Rest-Tour vom letzten Knoten aus

- diese Auswahl lagern wir nun wie beim Nearest Neighbor in eine Funktion aus:
  - Tipp: Ein Vergleich mit der Funktion `select_nearest_neighbor` zeigt die Ähnlichkeit beider Funktionen!

In [369]:
@njit
def select_using_rollout_nn(permutation, distance_matrix):
    
    node = permutation[len(permutation)-1] 
    best_q_value = 1000000
    best_node = node        
        
    for next_node in range(len(distance_matrix)):
        if next_node in permutation: 
            continue            
        # _, heißt, dass wir den ersten Rückgabewert ignorieren
        
        # Berechne die NN-Länge der Rest-Tour von next_node aus
        _, nn_value = tsp_nearest_neighbor(permutation + [next_node], distance_matrix)
        
        q_value = distance_matrix[node,next_node] + nn_value

        if q_value < best_q_value:
            best_node = next_node
            best_q_value = q_value

    
   
    return best_node, distance_matrix[node,best_node]


## Die Hauptfunktion:

In [372]:
@njit
def tsp_rollout_nn(permutation, distance_matrix):
        
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(permutation) < len(distance_matrix):    
        
        next_node, distance = select_using_rollout_nn(permutation, distance_matrix)
        permutation.append(next_node)
        total_distance += distance     
        
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

..probieren wir es wieder aus:

In [374]:
permutation, distance = tsp_rollout_nn([0], distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance)



Solution feasible, distance is:  8042


## Übung: Multistart Rollout


><div class="alert alert-block alert-info">
<b>Implementieren Sie eine Multistart-Rollout-Funktion! </b></div>  

In [ ]:
def tsp_multistart_nearest_neighbor(distance_matrix):
    

## Simplified Rollout

- wir wollen nun auch die Idee des Simplified Rollout übertragen in die neue Form der Implementierung

><div class="alert alert-block alert-info">
<b>Wo müssen die größeren Änderungen durchgeführt werden - in der `select`-Hilfsfunktion oder in der Hauptfunktion?</b></div>  

## Simplified Rollout: Die Select-Funktion

In [429]:
@njit
def select_using_simplified_rollout_nn(permutation, distance_matrix,  max_number_of_neighbors_rollout):
    
    node = permutation[len(permutation)-1] 
                                    
    best_q_value = 1000000
    best_node = node
   
    sorted_neighbors = np.argsort(distance_matrix[node])
    
    number_of_neighbors_rollout = 0
    for next_node in sorted_neighbors:
        if next_node in permutation: 
            continue            
       
        
        number_of_neighbors_rollout += 1
        # zähler wenn 
        if number_of_neighbors_rollout > max_number_of_neighbors_rollout:
            break     
                                    
                
        _, nn_value = tsp_nearest_neighbor(permutation + [next_node], distance_matrix)
        
        q_value = distance_matrix[node,next_node] + nn_value

        if q_value < best_q_value:
            best_node = next_node
            best_q_value = q_value

    return best_node, distance_matrix[node,best_node]



## Simplified Rollout: Die Hauptfunktion

In [430]:
@njit
def tsp_simplified_rollout_nn(permutation, distance_matrix, max_number_of_neighbors_rollout):
        
    total_distance = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(permutation) < len(distance_matrix):    
        
        next_node, distance = select_using_simplified_rollout_nn(permutation, distance_matrix, max_number_of_neighbors_rollout)
        permutation.append(next_node)
        total_distance += distance     
        
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

...probieren wir es aus:

In [433]:
max_number_of_neighbors_rollout = 3
permutation, distance = tsp_simplified_rollout_nn([0], distance_matrix, max_number_of_neighbors_rollout)
print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance)


Solution feasible, distance is:  8687


## Rollout als *one-step-lookahead*

Wir wollen uns nun folgende Interpretation des Rollout-Algorithmus anschauen:
- beim Rollout machen wir eine Art "Vorschau" (*lookahead*):
- anstatt nur auf den *kurzfristigen Effekt* zu schauen, wird die Qualität der "Rest-Tour" mittels Nearest Neighbor *abgeschätzt* und aus beiden der *Q-Faktor* / *Q-Wert* gebildet
- der Schritt, bei dem wir das Minimum über alle zulässigen Nachbarn betrachten, wird manchmal auch *one-step lookahead minimization* genannt
 - bei uns passiert das in der Funktion `select_using_rollout_nn`




## Varianten des Rollout

Allgemein kann man sagen, dass beim Rollout in der "lookahead-Schleife" ein *scnheller, heuristischer Algorithmus* zur Lösung des Restproblems aufgerufen wird
- je besser diese Abschätzung, desto besser sollte die letztendliche Gesamtlösung werden

Dies führt zu einer Reihe von Varianten dieser Idee, 
- Rollout mit mehreren Heuristiken / Greedy-Verfahren, die zu einer *super-heuristic* zusammengefasst werden
  - verschiedene Algorithmen werden auf das Restproblem angewendet
  - der beste Zielfunktionswert wird zur Abschätzung genutzt
- *Multi-Step-Lookahead*
  - dabei wird der gesamte Rollout-Algorithmus (inkl. lookahead-Minimierung) zur Abschätzung genutzt
  - das schauen wir uns jetzt einmal an




## Von One-Step zu Two-Step Lookahead

- anstatt bei der Auswahl des nächsten Knotens direkt die Heuristik (z.B. nearest neighbor) laufen zu lassen
- nutzen wir den kompletten Rollout-Algorithmus (inkl. one-step lookahead)

<img src="./img/mulistep_lookahead.png" width="60%">


## Von One-Step zu Two-Step Lookahead

- anstatt bei der Auswahl des nächsten Knotens direkt die Heuristik (z.B. nearest neighbor) laufen zu lassen
- nutzen wir den kompletten Rollout-Algorithmus (inkl. one-step lookahead)
- die Implementierung der entsprechenden "Select"-Funktion ist relativ einfach...


><div class="alert alert-block alert-info">
<b>Vervollständigen Sie die Implementierung unten! </b></div>  



In [427]:
@njit
def select_using_two_step_lookahead(permutation, distance_matrix):
    node = permutation[-1] 
    best_q_value = 1000000
    best_node = node
    
    for next_node in range(len(distance_matrix)):
        if next_node in permutation: 
            continue            
           

    
   
    return best_node, distance_matrix[node,best_node]    

## Two-Step Lookahead: Die Haupt-Funktion

Beachte: Wenn nur noch ein Knoten übrig ist, macht Two-Step-Lookahead keinen Sinn mehr!

In [426]:
@njit
def tsp_rollout_nn_two_step_lookahead(permutation, distance_matrix):
    

    total_distance = 0
    
    node = permutation[len(permutation)-1]
    
    #Beachte: Two-Step Lookahead bei nur noch einem Knoten ist nicht sinnvoll
    while len(permutation) < len(distance_matrix)-1:    
        
        if len(permutation) < len(distance_matrix) -1:
            node, distance = select_using_simplified_two_step_lookahead(permutation, distance_matrix, max_number_of_neighbors)
        else:
            node, distance = select_using_rollout_nn(permutation, distance_matrix)
            
                    
        permutation.append(node)
        total_distance += distance     
    
    
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

In [393]:
permutation, distance = tsp_rollout_nn_two_step_lookahead([0], distance_matrix)
print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance)


C:\Users\Michael\AppData\Local\Temp\ipykernel_10904\2292363227.py:12: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'select_using_two_step_lookahead'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\AppData\Local\Temp\ipykernel_10904\2838914185.py", line 1:
<source missing, REPL/exec in use?>

  node, distance = select_using_two_step_lookahead(permutation, distance_matrix)
C:\Users\Michael\miniconda3\envs\audproj\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'permutation' of function 'tsp_rollout_nn_two_step_lookahead'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#d

Solution feasible, distance is:  7902


## Die "Simplified-Variante"

In [398]:
@njit
def select_using_simplified_two_step_lookahead(permutation, distance_matrix, max_number_of_neighbors):
    node = permutation[-1] 
    best_q_value = 1000000
    best_node = node
    
    sorted_neighbors = np.argsort(distance_matrix[node])
    
    number_of_neighbors_rollout = 0
    for next_node in sorted_neighbors:
        if next_node in permutation: 
            continue            
       
        
        number_of_neighbors_rollout += 1
        # zähler wenn 
        if number_of_neighbors_rollout > max_number_of_neighbors:
            break            
            # _, heißt, dass wir den ersten Rückgabewert ignorieren


        _, value = tsp_rollout_nn(permutation + [next_node], distance_matrix)


        q_value = distance_matrix[node,next_node] + value

        if q_value < best_q_value:
            best_node = next_node
            best_q_value = q_value
    
   
    return best_node, distance_matrix[node,best_node]

    

## Die "Simplified-Variante"

In [434]:
@njit
def tsp_rollout_nn_simplified_two_step_lookahead(permutation, distance_matrix, max_number_of_neighbors):
    

    total_distance = 0
    
    node = permutation[len(permutation)-1]
    
 
    while len(permutation) < len(distance_matrix):   
        
        #Beachte: Two-Step Lookahead bei nur noch einem Knoten ist nicht sinnvoll
        if len(permutation) < len(distance_matrix) -1:
            node, distance = select_using_simplified_two_step_lookahead(permutation, distance_matrix, max_number_of_neighbors)
        else:
            node, distance = select_using_simplified_rollout_nn(permutation, distance_matrix, max_number_of_neighbors)
            
        permutation.append(node)
        total_distance += distance     
    

    
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

In [402]:
max_number_of_neighbors = 5
permutation, distance = tsp_rollout_nn_simplified_two_step_lookahead([0], distance_matrix, 5)
print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance)


Solution feasible, distance is:  7897


## Von Two-Step zu Multi-Step-Lookahead

**Kernidee:**
- wir parametrisieren die Anzahl der Lookahead-Schritte
- und rufen den lookahead-Algorithmus rekursiv mit jeweils um 1 reduzierter Anzahl an Lookahead-Schritten auf!

- hier direkt die Simplified-Variante

In [413]:
@njit
def select_using_simplified_multi_step_lookahead(permutation, distance_matrix, max_number_of_neighbors, number_of_lookahead_steps):
    node = permutation[-1] 
    best_q_value = 1000000
    best_node = node
    
    number_of_lookahead_steps = min(number_of_lookahead_steps, len(distance_matrix) - len(permutation))
    
    sorted_neighbors = np.argsort(distance_matrix[node])
    
    number_of_neighbors_rollout = 0
    
    for next_node in sorted_neighbors:
        if next_node in permutation: 
            continue            
       
        
        number_of_neighbors_rollout += 1
        # zähler wenn 
        if number_of_neighbors_rollout > max_number_of_neighbors:
            break                     
            

        if number_of_lookahead_steps > 1:
            _, value = tsp_rollout_nn_simplified_multi_step_lookahead(permutation + [next_node],
                                                                      distance_matrix, 
                                                                      max_number_of_neighbors,
                                                                      number_of_lookahead_steps - 1)
        else:
            _, value = tsp_rollout_nn(permutation + [next_node], distance_matrix)

        q_value = distance_matrix[node,next_node] + value

        if q_value < best_q_value:
            best_node = next_node
            best_q_value = q_value
    
   
    return best_node, distance_matrix[node,best_node]

    

In [422]:
@njit
def tsp_rollout_nn_simplified_multi_step_lookahead(permutation, distance_matrix, max_number_of_neighbors, number_of_lookahead_steps):
    

    total_distance = 0
    
    node = permutation[len(permutation)-1]    

    while len(permutation) < len(distance_matrix):    
        
        node, distance = select_using_simplified_multi_step_lookahead(permutation,
                                                                      distance_matrix,
                                                                      max_number_of_neighbors,
                                                                      number_of_lookahead_steps)
        permutation.append(node)
        total_distance += distance     
     
    
    total_distance += distance_matrix[permutation[len(permutation)-1],permutation[0]]
    return permutation, total_distance

In [425]:
max_number_of_neighbors = 3
number_of_lookahead_steps = 2
permutation, distance = tsp_rollout_nn_simplified_multi_step_lookahead([2], 
                                                                       distance_matrix,
                                                                       max_number_of_neighbors,
                                                                       number_of_lookahead_steps)
print_obj_and_eval_tsp_solution(distance_matrix, permutation, distance)

Solution feasible, distance is:  7923
